In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [2]:
base_dir = '../data/MagnaTagATune/'
annotation_fn = 'annotations_final.csv'
annotation_path = base_dir + annotation_fn
annotation_out_fn = 'annotation_reduced_50.csv'
annotation_out_path = base_dir + annotation_out_fn

annotations = pd.read_csv(annotation_path, sep='\t')

In [3]:
print("Len before removing wrong songs:{}".format(len(annotations)))
to_be_removed = ["6/norine_braun-now_and_zen-08-gently-117-146.mp3", 
                 "8/jacob_heringman-josquin_des_prez_lute_settings-19-gintzler__pater_noster-204-233.mp3",
                 "9/american_baroque-dances_and_suites_of_rameau_and_couperin-25-le_petit_rien_xiveme_ordre_couperin-88-117.mp3"]

annotations = annotations.drop(index = annotations.loc[annotations['mp3_path'].isin(to_be_removed)].index)\
                                                                                        .reset_index(drop=True)
print("Len after removing wrong songs:{}".format(len(annotations)))

Len before removing wrong songs:25863
Len after removing wrong songs:25860


In [4]:
synonyms = [['drum','beat', 'beats','drums'],['chant', 'chanting'],['choir', 'choral'],
                ['classical', 'clasical', 'classic'],
                ['electro', 'electronic', 'electronica', 'electric'],
                ['fast', 'fast beat', 'quick'],
                ['female', 'female singer', 'female singing', 'female vocals','female vocal','female voice', 'woman', 'woman singing', 'women'],
                ['flute', 'flutes'],['guitar', 'guitars'],['hard', 'hard rock'],['harpsichord', 'harpsicord'],
                ['heavy', 'heavy metal', 'metal'],['horn', 'horns'],['india', 'indian'],['jazz', 'jazzy'],
                ['male', 'male singer', 'male vocal', 'male vocals', 'male voice', 'man', 'man singing', 'men'],
                ['opera', 'operatic'],['orchestra', 'orchestral'],['quiet', 'silence'],['singer', 'singing'],
                ['space', 'spacey'],['string', 'strings'],['synth', 'synthesizer'],['violin', 'violins'],
                ['vocal', 'vocals', 'voice', 'voices'],['strange', 'weird']]

non_relevant_tags = ['no flute', 'no guitar', 'no piano','no strings', 
                     'no violin','not classical', 'not english','not opera', 
                     'not rock','no beat', 'no singer','no drums', 'no singing',
                     'no vocal', 'no vocals', 'no voice', 'no voices']

annotations.drop(columns=non_relevant_tags, inplace=True)

#each tag is mapped to the first synonym
renaming_map = {}
for row in synonyms:
    ref = row[0]
    for syn in row:
        if syn != ref:
            renaming_map[syn] = ref

In [5]:
#rename and merge synonym columns
annotations.rename(columns=renaming_map, inplace=True)
annotations = annotations.groupby(lambda x:x, axis=1).sum()
annotations = annotations.applymap(lambda x: 1 if type(x)==int and x>=1 else x)
print("After merge: {}".format(len(annotations.columns.values)))

After merge: 125


In [6]:
n = 50
top_n = (annotations.drop(['clip_id', 'mp3_path'], axis=1)
               .sum()
               .sort_values()
               .tail(n)
               .index
               .tolist())
top_n_values =  (annotations.drop(['clip_id', 'mp3_path'], axis=1)
               .sum()
               .sort_values()
               .tail(n)
               .values)

In [7]:
top_n_values

array([ 217,  225,  241,  243,  253,  275,  296,  296,  312,  323,  327,
        337,  384,  406,  541,  555,  575,  583,  622,  640,  649,  650,
        791,  826,  926,  985,  995, 1034, 1072, 1086, 1122, 1298, 1308,
       1402, 1734, 1906, 1956, 2056, 2067, 2169, 2330, 2371, 2764, 2813,
       2840, 2954, 3547, 3813, 4356, 4860])

In [12]:
# plot 
from bokeh.io import show, output_file
from bokeh.plotting import figure
import bokeh

output_path = 'figures/tags_distribution.png'

p = figure(plot_height=350, x_axis_label = 'tags', y_axis_label = 'count',
           tools="save")


p.vbar(x=np.arange(0,len(top_n)), top=top_n_values, width=0.9)

p.xgrid.grid_line_color = None
p.y_range.start = 0
show(p)

In [ ]:
total_tag_instances = annotations.drop(['clip_id','mp3_path'],axis=1).sum(axis=0).sum()
total_tag_instances_top_n = annotations[top_n].sum(axis=0).sum()
print("Coverage: {}".format(total_tag_instances_top_n/total_tag_instances))

In [ ]:
final_annotations = annotations[top_n + ['mp3_path']] 
final_annotations.to_csv(path_or_buf=annotation_out_path, sep='\t')